In [25]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ../Model.ipynb
%run ../Dataset.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/05-MNIST/02-UAE_for_MNIST


In [26]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device
        
        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []
    
    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()
        
        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for x, x_, y in train_loader:  # label kullanılmıyor
                x = x.to(self.device)
                x_ = x_.to(self.device)
                
                z_hat_, x_hat_ = self.model(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,784))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation
            if val_loader:
                val_loss = self.validate(val_loader)
                
                # Early stopping kontrolü
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0
        
                else:
                    self.patience = self.patience + 1
        
                if self.patience > self.max_patience:
                    break
                    
            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))
            
    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        
        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)
                
                z_hat_, x_hat_ = self.model(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,784))
                
                total_loss += loss.item()
                
        avg_loss = total_loss / len(val_loader)
        #print(f"→ Validation Loss: {avg_loss:.6f}")
        self.model.train()
        return avg_loss

In [27]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=784):

        self.normal_dist = split_ratio*np.random.randn(dim,)
        
    def __call__(self, x):
      return x + self.normal_dist

In [28]:
# Hyper-Parameters & Settings

batch_size = 1000
lr = 0.001

epochs = 200
max_patience = 200

split_ratio = 0.0001

In [29]:
# Dataset
train_dataset = MNISTDataset(mode='train', transform=NoiseTransform(split_ratio))
val_dataset = MNISTDataset(mode='val', transform=NoiseTransform(split_ratio))
test_dataset = MNISTDataset(mode='test')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [30]:
"""
X_list = []
y_list = []

for x, _, y in train_dataset:
    X_list.append(x)
    y_list.append(y)

X = torch.stack(X_list)
y = torch.tensor(y_list)

print('9 dan',X[y==9].shape[0], 'sayıda var')
"""

"\nX_list = []\ny_list = []\n\nfor x, _, y in train_dataset:\n    X_list.append(x)\n    y_list.append(y)\n\nX = torch.stack(X_list)\ny = torch.tensor(y_list)\n\nprint('9 dan',X[y==9].shape[0], 'sayıda var')\n"

In [31]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

# Model
name = 'results/UAE_MNIST'
model = To_Uniform(
                 encoder_layers=[784, 2000, 2000, 2000, 2000, 3],
                 decoder_layers=[3, 2000, 2000, 2000, 2000, 784],
                 encoder_act=nn.SiLU,
                 decoder_act=nn.SiLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [32]:
# Training
trainer = Trainer(model, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu', max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

saved!
Epoch   1 | Train Loss: 0.069952 | Validation Loss: 0.055738
saved!
Epoch   2 | Train Loss: 0.052645 | Validation Loss: 0.048323
saved!
Epoch   3 | Train Loss: 0.045943 | Validation Loss: 0.042402
saved!
Epoch   4 | Train Loss: 0.041840 | Validation Loss: 0.040205
saved!
Epoch   5 | Train Loss: 0.039141 | Validation Loss: 0.037783
saved!
Epoch   6 | Train Loss: 0.036669 | Validation Loss: 0.036122
saved!
Epoch   7 | Train Loss: 0.035486 | Validation Loss: 0.034558
saved!
Epoch   8 | Train Loss: 0.034210 | Validation Loss: 0.033657
saved!
Epoch   9 | Train Loss: 0.033925 | Validation Loss: 0.033628
saved!
Epoch  10 | Train Loss: 0.033241 | Validation Loss: 0.032158
saved!
Epoch  11 | Train Loss: 0.031846 | Validation Loss: 0.031777
Epoch  12 | Train Loss: 0.031723 | Validation Loss: 0.032243
saved!
Epoch  13 | Train Loss: 0.031455 | Validation Loss: 0.031178
saved!
Epoch  14 | Train Loss: 0.031727 | Validation Loss: 0.030355
saved!
Epoch  15 | Train Loss: 0.030450 | Validation Lo

In [33]:
# CSV dosyasına kaydet
train_losses = trainer.train_cost  # liste veya numpy array
val_losses = trainer.val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)), 
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")